# Práctica 9 - Embedding LSTM

> **Intituto Politécnico Nacional**
>
> **Centro de Investigación en Computación**
>
> 
> 
> Departamento de Diplomados y Extensión Profesional
> 
> Diplomado en Inteligencia Artificial
> 
> * Módulo 11 - Parte II
> 
> Profesor: Alan Badillo Salas (badillosalas@outlook.com)

In [15]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [16]:
text_only_train_ds = train_ds.map(lambda x, y: x)

In [ ]:
max_length = 600
max_tokens = 500

text_vectorization = keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)

text_vectorization.adapt(text_only_train_ds)

2025-09-04 17:16:14.708014: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [18]:
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [19]:
inputs = keras.Input(shape=(None,), dtype="int64")

embedded = keras.layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)

x = keras.layers.Bidirectional(keras.layers.LSTM(32))(embedded)
x = keras.layers.Dropout(0.5)(x)

outputs = keras.layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(
    optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, None, 256)      │       128,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 64)             │        73,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 202,049 (789.25 KB)

 Trainable params: 202,049 (789.25 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
callbacks = [
    keras.callbacks.ModelCheckpoint("resultados/embeddings_bidir_lstm.keras", save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 90s 142ms/step - accuracy: 0.6401 - loss: 0.6287 - val_accuracy: 0.7086 - val_loss: 0.5592
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 86s 138ms/step - accuracy: 0.7574 - loss: 0.5286 - val_accuracy: 0.8004 - val_loss: 0.4618
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 84s 134ms/step - accuracy: 0.7950 - loss: 0.4751 - val_accuracy: 0.8210 - val_loss: 0.4138
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 83s 133ms/step - accuracy: 0.8185 - loss: 0.4287 - val_accuracy: 0.8278 - val_loss: 0.3927
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 132ms/step - accuracy: 0.8255 - loss: 0.4130 - val_accuracy: 0.8380 - val_loss: 0.3875
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 85s 135ms/step - accuracy: 0.8353 - loss: 0.3947 - val_accuracy: 0.8382 - val_loss: 0.4051
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 85s 136ms/step - accuracy: 0.8436 - loss: 0.3760 - val_accuracy: 0.8366 - val_loss: 0.3739
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 90s 145ms/step - accuracy: 0.8515 - loss: 0

In [21]:
model = keras.models.load_model("resultados/embeddings_bidir_lstm.keras", safe_mode=False)
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 27s 34ms/step - accuracy: 0.8390 - loss: 0.3790
Test acc: 0.839
